## Load Model From huggingface

In [1]:
import os
import tqdm
import joblib
import numpy as np
import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModel

checkpoint = "THUDM/chatglm-6b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True)


/home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.10s/it]


## Insert LoRA to model

In [2]:
import peft
import loralib as lora
from peft import LoraConfig


config = LoraConfig(
              peft_type="LORA", 
              task_type="SEQ_2_SEQ_LM", 
              r=32, 
              lora_alpha=32, 
              target_modules=["q", "k", "v"],
              lora_dropout=0.1, 
              )


class QKV_layer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(QKV_layer, self).__init__()
        self.linear_q = torch.nn.Linear(in_features, out_features//3)
        self.linear_k = torch.nn.Linear(in_features, out_features//3)
        self.linear_v = torch.nn.Linear(in_features, out_features//3)

    def update(self, target_layer):
        self.linear_q.weight.data = target_layer.weight[:target_layer.out_features//3, :].data
        self.linear_q.bias.data = target_layer.bias[:target_layer.out_features//3].data

        self.linear_k.weight.data = target_layer.weight[target_layer.out_features//3:target_layer.out_features//3*2, :].data
        self.linear_k.bias.data = target_layer.bias[target_layer.out_features//3:target_layer.out_features//3*2].data

        self.linear_v.weight.data = target_layer.weight[target_layer.out_features//3*2:, :].data
        self.linear_v.bias.data = target_layer.bias[target_layer.out_features//3*2:].data
    
    def forward(self, x):
        q = self.linear_q(x)
        k = self.linear_k(x)
        v = self.linear_v(x)
        return torch.concat([q,k,v], dim = -1)


for key, module in model.named_modules():
    if key.endswith('attention'):
        try:
            # Here we split the query_key_value layer into three linear layer for LoRA. But you can also use merged linear.
            qkv_layer = QKV_layer(module.query_key_value.in_features, module.query_key_value.out_features) 
            qkv_layer.update(module.query_key_value)
            module.query_key_value = qkv_layer
        except:
            pass
        module.query_key_value = peft.tuners.lora.LoraModel(config, module.query_key_value)


lora.mark_only_lora_as_trainable(model)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
trainable_params = sum([np.prod(p.size()) for p in model_parameters])

model_parameters = filter(lambda p: not p.requires_grad, model.parameters())
non_trainable_params = sum([np.prod(p.size()) for p in model_parameters])


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116_nocublaslt.so...


/home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/jiangjin/miniconda3/envs/visgpt did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [3]:
print('trainable_params:{} ({:.2f}%), non_trainable_params:{}'.format(trainable_params, trainable_params/non_trainable_params*100,non_trainable_params))

trainable_params:22020096 (0.36%), non_trainable_params:6173286400


## Dataset

In [4]:
import json
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

device = 'cuda'
EOS_ID = 130005
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

with open('data/alpaca_data.json', 'r') as f:
    content = json.load(f)


pairs = []

for line in content:
    if line['input'] == '':
        prompt = PROMPT_DICT['prompt_no_input'].format_map(line)
    else:
        prompt = PROMPT_DICT['prompt_input'].format_map(line)
    completion = line['output']
    pairs.append({'prompt':prompt, 'completion':completion})


class AlpacaDataset(Dataset):
    def __init__(self, pairs, tokenizer) -> None:
        super().__init__()
        self.pairs = pairs
        self.tokenizer = tokenizer
 
    def __getitem__(self, index):
        prompt = self.tokenizer.encode(self.pairs[index]['prompt'])
        completion = self.tokenizer.encode(self.pairs[index]['completion'], add_special_tokens=False) + [EOS_ID]

        seq = prompt + completion
        context_length = seq.index(130004) + 1

        attention_mask = torch.ones((len(seq), len(seq)), device=device)
        attention_mask.tril_()
        attention_mask[..., :context_length - 1] = 1
        attention_mask.unsqueeze_(0)
        attention_mask = (attention_mask < 0.5).bool()

        position_ids = torch.stack([torch.arange(0,len(seq), device=device), torch.concat([torch.zeros(context_length-2, device=device), torch.arange(0,len(seq)-context_length+2, device=device)])]).long()
        labels = torch.tensor([-100] * len(prompt) + completion, device=device).long()

        return {'input_ids':seq, 'attention_mask':attention_mask, "labels":labels, 'position_ids':position_ids}

    def __len__(self):
        return len(self.pairs)



def collate_fn(batch):
    input_ids = []
    attention_mask = []
    labels = []
    position_ids = []
    # TODO: padding for batch training
    for obj in batch:
        input_ids.append(obj['input_ids'])
        attention_mask.append(obj['attention_mask'])
        labels.append(obj['labels'])
        position_ids.append(obj['position_ids'])
    return {'input_ids': torch.tensor(input_ids).long(), 
            'attention_mask': torch.stack(attention_mask), 
            'labels': torch.stack(labels),
            'position_ids':torch.stack(position_ids)}

            

train_dataset = AlpacaDataset(pairs,tokenizer=tokenizer,)
train_dataloader = DataLoader(dataset=train_dataset, collate_fn = collate_fn, shuffle=True, batch_size=1)

## Training

Here we show how to train the model with gradient accumulation as well as mix precision, and then save the model (only LoRA's weight which is typically within 10MB).

In [5]:
from torch.cuda.amp import autocast
from transformers import get_linear_schedule_with_warmup

LR = 2e-5
NUM_EPOCHS = 2
accumulate_step = 32
version = 'test'

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=int(len(train_dataloader) / accumulate_step),
    num_training_steps=(int(len(train_dataloader) / accumulate_step) * NUM_EPOCHS),
)


model.to(device).train()

with autocast(dtype=torch.float16):
    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for step, batch in enumerate(t:=tqdm.tqdm(train_dataloader)):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss_d = outputs.loss.detach().float()
            t.set_description(f"loss: {loss_d}")
            total_loss += loss_d
            loss = outputs.loss / accumulate_step
            loss.backward()
            if (step+1) % accumulate_step == 0:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        peft_model_id = f"{checkpoint}_{version}_{epoch}"
        torch.save(lora.lora_state_dict(model), peft_model_id+'.pt')
        print(epoch, total_loss/(step+1))
    

loss: 2.0446531772613525:   2%|▏         | 1226/52002 [03:51<2:39:43,  5.30it/s] 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 │   │   total_loss = 0                                                                      │
│   23 │   │   for step, batch in enumerate(t:=tqdm.tqdm(train_dataloader)):                       │
│   24 │   │   │   batch = {k: v.to(device) for k, v in batch.items()}                             │
│ ❱ 25 │   │   │   outputs = model(**batch)                                                        │
│   26 │   │   │   loss_d = outputs.loss.detach().float()                                          │
│   27 │   │   │   t.set_description(f"loss: {loss_d}")                                            │
│   28 │   │   │   total_loss += loss_d                                                            │
│                                                                                                  │
│ /home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/torch/nn/modules/module.py:119 │
│ 4 in _call_impl                                                                                  │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b/969290547e761b20fdb96b060 │
│ 2b4fd8d863bbb85/modeling_chatglm.py:1158 in forward                                              │
│                                                                                                  │
│   1155 │   │   use_cache = use_cache if use_cache is not None else self.config.use_cache         │
│   1156 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1157 │   │                                                                                     │
│ ❱ 1158 │   │   transformer_outputs = self.transformer(                                           │
│   1159 │   │   │   input_ids=input_ids,                                                          │
│   1160 │   │   │   position_ids=position_ids,                                                    │
│   1161 │   │   │   attention_mask=attention_mask,                                                │
│                                                                                                  │
│ /home/jiangjin/miniconda3/envs/visgpt/lib/python3.8/site-packages/torch/nn/modules/module.py:119 │
│ 4 in _call_impl                                                                                  │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)   

: 

## Inference

In [ ]:
# reload the model
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

# convert it again
for key, module in model.named_modules():
    if key.endswith('attention'):
        try:
            qkv_layer = QKV_layer(module.query_key_value.in_features, module.query_key_value.out_features) 
            qkv_layer.update(module.query_key_value)
            module.query_key_value = qkv_layer
        except:
            pass
        module.query_key_value = peft.tuners.lora.LoraModel(config, module.query_key_value)


# load the LoRA checkpoint
model.load_state_dict(torch.load('.pt file you saved'), strict=False)

model.half().cuda().eval()

# Let's chat!
response, history = model.chat(tokenizer, "Hello", history=[])